In [1]:
import ipywidgets as widgets
import pandas as pd
import pm_lib

from IPython.display import SVG, display

In [2]:
selection_fa = ['NFA','DFA']

wid_selection_fa = widgets.Dropdown(name="FA")
wid_selection_fa.options = selection_fa

In [11]:
col_names = ['Word', 'Expected']

def fa_selection(fa_type):
    if wid_selection_fa.value == 'NFA':
        arquivo = 'nfa.jff'
        arquivoTeste = 'nfa.cases'
        doc = 'nfa.cases'
        file = 'test_nfa'
        print("NFA example: ")
        nfa = pm_lib.jffToNFA(arquivo)
        pm_lib.nfa_to_graph(nfa)
        pm_lib.save_nfa_to_graph(nfa, file)
        display(SVG(file+'.svg'))
        data = pd.read_csv(doc, sep='\t', dtype=object, names = col_names)
        display(data)
        pm_lib.uncouple(pm_lib.nfa_validate(nfa, arquivoTeste))
    else:
        arquivo = 'dfa.jff'
        arquivoTeste = 'dfa.cases'
        doc = 'dfa.cases'
        file = 'test_dfa'
        print("DFA example: ")
        dfa = pm_lib.jffToDFA(arquivo)
        pm_lib.dfa_to_graph(dfa)
        pm_lib.save_dfa_to_graph(dfa, file)
        display(SVG(file+'.svg'))
        data = pd.read_csv(doc, sep='\t', dtype=object, names = col_names)
        display(data)
        pm_lib.uncouple(pm_lib.dfa_validate(dfa, arquivoTeste))
    
interactive_sel = widgets.interactive(fa_selection, fa_type=wid_selection_fa)

In [12]:
interactive_sel

interactive(children=(Dropdown(description='fa_type', index=1, options=('NFA', 'DFA'), value='DFA'), Output())…